In [1]:
import numpy as np
import pandas as pd
import scipy as sp

from statsmodels.stats.multitest import fdrcorrection

import cobra, cobra_utils

# Example of using reporter metabolites in *E. coli*

## Data Location

In [2]:
model_filename = '../data/e_coli_iML1515.xml.gz'

In [3]:
gene_expression_filename = '../data/EcoliExpression_GSE54900.xlsx'

## Load data

**Metabolic model**

In [4]:
model = cobra_utils.io.load_model(model_filename, format='sbml')

Loading genome-scale model
Model correctly loaded.


**Gene expression**

In [5]:
gene_expression = pd.read_excel(gene_expression_filename, index_col='Gene')

In [6]:
gene_expression.head()

,wt_dpd_1,wt_dpd_2,delfur_dpd_1,delfur_dpd_2
Gene,,,,
b0002,8.113347,8.240386,6.915225,7.133376
b0003,7.918491,8.181859,6.652128,7.064316
b0004,7.248520,7.471647,6.059338,6.295427
b0005,3.758424,3.698816,3.207263,3.296815
b0006,4.696611,4.701444,4.635023,4.755118


## Get differential expession for two conditions

**p-value of differential expression**

In [7]:
gene_expression['p-value'] = gene_expression.apply(lambda row: sp.stats.ttest_ind(row[0:2], row[2:4])[1], axis=1)

**FDR**

In [8]:
rej, adj_p = fdrcorrection(gene_expression['p-value'].fillna(1.).values, alpha=0.05, is_sorted=False)

In [9]:
gene_expression['corrected p-value'] = adj_p

In [10]:
gene_expression.head()

,wt_dpd_1,wt_dpd_2,delfur_dpd_1,delfur_dpd_2,p-value,corrected p-value
Gene,,,,,,
b0002,8.113347,8.240386,6.915225,7.133376,0.011782,0.402468
b0003,7.918491,8.181859,6.652128,7.064316,0.039616,0.402468
b0004,7.248520,7.471647,6.059338,6.295427,0.018343,0.402468
b0005,3.758424,3.698816,3.207263,3.296815,0.012499,0.402468
b0006,4.696611,4.701444,4.635023,4.755118,0.953493,0.986424


## Reporter metabolites

**Compute reporter metabolites**

In [11]:
rep_met = cobra_utils.topology.reporter_metabolites(model, gene_expression[['corrected p-value']])

Running reporter metabolites analysis
Getting information for all metabolites in the model.
Information correctly obtained.


**Top 20 reporter metabolites**

In [12]:
rep_met.head(20)

,p-value,corrected Z,mean Z,std Z,gene number
h_c,0.001015,3.085931,-0.524420,1.097564,885.0
glu__L_c,0.003567,2.690531,-0.195846,0.592217,56.0
h2o_c,0.003854,2.664570,-0.511701,0.952951,551.0
atp_c,0.004910,2.582137,-0.477499,1.026978,342.0
adp_c,0.005580,2.537653,-0.471624,1.056136,309.0
pi_c,0.019712,2.059733,-0.508153,1.066590,315.0
ptrc_p,0.023374,1.988571,-0.071768,0.429209,19.0
gln__L_c,0.026748,1.930893,-0.128925,0.401942,22.0
glc__D_p,0.028584,1.902023,-0.146143,0.681053,23.0
fadh2_c,0.031439,1.860055,-0.014573,0.338252,14.0


**Information of the reactions associated to the top-16 reporter metabolites**

In [13]:
rxn_info = cobra_utils.query.rxn_info_from_metabolites(model, list(rep_met.head(16).index))

Using list of metabolites to get reactions where they participate. Also, getting genes of those reactions.
Information correctly obtained.


/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/cobra_utils/cobra_utils/query/rxn_info.py:40 UserWarning: set() are not in the model


In [14]:
rxn_info

,MetID,MetName,RxnID,RxnName,GeneID,Subsystem,RxnFormula
0,4abut_c,4-Aminobutanoate,GLUABUTt7pp,4-aminobutyrate/glutamate antiport (periplasm),b1492,,4abut_c + glu__L_p <=> 4abut_p + glu__L_c
1,4abut_c,4-Aminobutanoate,GLUDC,Glutamate Decarboxylase,b3517,,glu__L_c + h_c --> 4abut_c + co2_c
2,4abut_c,4-Aminobutanoate,GLUDC,Glutamate Decarboxylase,b1493,,glu__L_c + h_c --> 4abut_c + co2_c
3,4abut_c,4-Aminobutanoate,GGGABAH,Gamma-glutamyl-gamma-aminobutyric acid hydrolase,b1298,,gg4abut_c + h2o_c --> 4abut_c + glu__L_c
4,4abut_c,4-Aminobutanoate,ABTA,4-aminobutyrate transaminase,b2662,,4abut_c + akg_c --> glu__L_c + sucsal_c
...,...,...,...,...,...,...,...
5833,fad_c,Flavin adenine dinucleotide oxidized,ACOAD1f,Acyl-CoA dehydrogenase (butanoyl-CoA),b0221,,btcoa_c + fad_c --> b2coa_c + fadh2_c
5834,fad_c,Flavin adenine dinucleotide oxidized,DAAD,D-Amino acid dehydrogenase,b1189,,ala__D_c + fad_c + h2o_c --> fadh2_c + nh4_c +...
5835,fad_c,Flavin adenine dinucleotide oxidized,ARBTNR1,Aerobactin reductase,,,2.0 arbtn_fe3_c + fadh2_c --> 2.0 arbtn_c + fa...
5836,fad_c,Flavin adenine dinucleotide oxidized,FADRx2,FAD reductase,b2763,,fad_c + h_c + nadph_c --> fadh2_c + nadp_c


## Reporter pathways

This model does not contain information about subsystems. Simulate random pathways

In [15]:
import random

rxn_pathways_association = dict()
rxns_ = cobra_utils.query.get_rxn_ids(model)
number_pathways = 10
random.shuffle(rxns_)
split_rxns = np.array_split(rxns_, number_pathways)

for i in range(number_pathways):
    rxn_pathways_association['Pathway{}'.format(i+1)] = split_rxns[i].tolist()
    

**Compute reporter pathways**

In [16]:
genes_in_model = cobra_utils.query.get_gene_ids(model)

In [17]:
rep_pathways = cobra_utils.topology.reporter_pathways(model,
                                                      gene_expression.loc[gene_expression.index.isin(genes_in_model),
                                                                          ['corrected p-value']],
                                                      rxn_pathways_association=rxn_pathways_association
                                                     )

Running reporter pathways analysis


**Top 20 reporter pathways**

In [18]:
rep_pathways.head(20)

,p-value,corrected Z,mean Z,std Z,gene number
Pathway9,0.095795,1.305889,-0.466886,0.700554,288.0
Pathway3,0.183164,0.903373,-0.492649,0.746314,300.0
Pathway4,0.296459,0.534612,-0.514042,0.755820,282.0
Pathway5,0.342639,0.405273,-0.523138,1.118155,298.0
Pathway7,0.419479,0.203227,-0.534673,0.721534,274.0
Pathway8,0.557512,-0.144664,-0.556600,1.125027,281.0
Pathway1,0.559028,-0.148505,-0.557107,0.798160,266.0
Pathway2,0.599971,-0.253272,-0.563281,1.145849,286.0
Pathway10,0.691828,-0.501038,-0.580386,1.162774,258.0
Pathway6,0.960144,-1.752361,-0.652387,1.592258,307.0
